#

In [1]:
# TODO: extend final template size?

In [2]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
# This draws matches and optionally a set of inliers in a different color

# include the drawMatches function
def drawMatches(img1, kp1, img2, kp2, matches, inliers = None):
    # Create a new output image that concatenates the two images together
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1,:] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:cols1+cols2,:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns, y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        inlier = False

        if inliers is not None:
            for i in inliers:
                if i.item(0) == x1 and i.item(1) == y1 and i.item(2) == x2 and i.item(3) == y2:
                    inlier = True

        # Draw a small circle at both co-ordinates
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points, draw inliers if we have them
        if inliers is not None and inlier:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 255, 0), 1)
        elif inliers is not None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 0, 255), 1)

        if inliers is None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)

    return out




In [4]:
#
# Runs sift algorithm to find features
#

def findFeatures(img):
    print("Finding Features...")
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    img = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite('sift_keypoints.png', img)

    return keypoints, descriptors

#
# Matches features given a list of keypoints, descriptors, and images
#
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    print("Matching Features...")
    matcher = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = matcher.match(desc1, desc2)
#     matchImg = drawMatches(img1,kp1,img2,kp2,matches)
#     cv2.imwrite('Matches.png', matchImg)
    return matches

In [5]:
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)

    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h


#
#Calculate the geometric distance between estimated points and original points
#
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)




In [6]:
#
#Runs through ransac algorithm, creating homographies from random correspondences
#
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    random.seed(2)
    for i in range(500):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            if d < 10:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        # print ("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break
    return finalH, maxInliers




In [8]:
# import os
# import matplotlib.pyplot as plt
# path = 'data'
# imgs = os.listdir(path)
# images = []
# for i, img in enumerate(imgs):
#     file_path = os.path.join(path, img)
#     images.append(plt.imread(file_path))
# plt.figure(figsize=(20,10))
# for i in range(len(images)):
#     plt.subplot(2,3,i+1)
#     plt.imshow(images[i])
#     plt.xlabel(imgs[i])
#     plt.xticks([])
#     plt.yticks([])

In [9]:
impairs = [
    ['./data/left.jpg', './data/center.jpg'],
    ['./data/up.jpg', './data/center.jpg'],
    ['./data/down.jpg', './data/center.jpg'],
    ['./data/right.jpg', './data/center.jpg'],]

In [ ]:
%%time
fig, axes = plt.subplots(len(impairs), 1, figsize=(10,40))
axes = axes.flatten()

for idx, img_names in enumerate(impairs):
    estimation_thresh = 0.5
    if estimation_thresh is None:
        estimation_thresh = 0.50

    img1name = str(img_names[0])
    img2name = str(img_names[1])

    img1_gray = cv2.imread(img1name, 0)
    img2_gray = cv2.imread(img2name, 0)

    img1 = cv2.imread(img1name)
    img2 = cv2.imread(img2name)
    #find features and keypoints
    correspondenceList = []
    if img1 is not None and img2 is not None:
        kp1, desc1 = findFeatures(img1)
        kp2, desc2 = findFeatures(img2)
        print ("Found keypoints in " + img1name + ": " + str(len(kp1)))
        print ("Found keypoints in " + img2name + ": " + str(len(kp2)))
        keypoints = [kp1,kp2]
        matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)

        for match in matches:
            (x1, y1) = keypoints[0][match.queryIdx].pt
            (x2, y2) = keypoints[1][match.trainIdx].pt
            correspondenceList.append([x2, y2, x1, y1])

        corrs = np.matrix(correspondenceList)

        #run ransac algorithm
        finalH, inliers = ransac(corrs, estimation_thresh)
        finalH = calculateHomography(inliers)
        

        final_shape = (2 * img1.shape[1] + img2.shape[1], 2 * img1.shape[0] + img2.shape[0])
        x_offset = (final_shape[0]  - img1.shape[1]) // 2
        y_offset = (final_shape[1] - img1.shape[0]) // 2
        finalH[0,2] += x_offset
        finalH[1,2] += y_offset
        
        
        dst = cv2.warpPerspective(src=img2,M=finalH,dsize=final_shape) #wraped image
        dst[y_offset:img1.shape[0]+y_offset, x_offset:img1.shape[1]+x_offset] = img1 #stitched image
        
        
        mask = dst.sum(axis=-1) >0
        dst = dst[np.ix_(mask.any(1),mask.any(0))]
    
        axes[idx].imshow(dst)
        axes[idx].set_title(img_names)

Finding Features...
Finding Features...
Found keypoints in ./data/left.jpg: 204
Found keypoints in ./data/center.jpg: 235
Matching Features...
Finding Features...
Finding Features...
Found keypoints in ./data/up.jpg: 306
Found keypoints in ./data/center.jpg: 235
Matching Features...
Finding Features...
Finding Features...
Found keypoints in ./data/down.jpg: 326
Found keypoints in ./data/center.jpg: 235
Matching Features...
Finding Features...
Finding Features...
Found keypoints in ./data/right.jpg: 149
Found keypoints in ./data/center.jpg: 235
Matching Features...
